In [2]:
from Bio import Entrez, SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import numpy as np
import pandas as pd
import re

import AS_motif_analysis as AS

In [3]:
# input significant alternative splicing (Skipped exons) evants with splicing sites coodinate
a3ss_sig = pd.read_csv('/home/yu/R_studio_work_dir/AS_further_analysis/output/A3SS_sig.csv')
a3ss_sig.iloc[:7,]

,ID,GeneID,geneSymbol,chr,strand,longExonStart_0base,longExonEnd,shortES,shortEE,flankingES,...,IncLevelDifference,IncLevel1-1,IncLevel1-2,IncLevel1-3,IncLevel1-4,IncLevel2-1,IncLevel2-2,IncLevel2-3,IncLevel2-4,AS_type
0,6302,ENSG00000144868,TMEM108,chr3,+,133229265,133229351,133229269,133229351,133045901,...,-0.754,0.000,0.000,0.000,0.140,0.662,1.000,1.000,0.495,A3SS
1,7536,ENSG00000130600,H19,chr11,-,1995175,1995794,1995175,1995788,1995875,...,-0.081,0.814,0.687,0.952,0.833,0.847,0.938,0.834,0.990,A3SS
2,2594,ENSG00000141994,DUS3L,chr19,-,5786748,5786919,5786748,5786845,5787060,...,0.268,0.402,0.183,0.278,0.473,0.000,0.119,0.000,0.144,A3SS
3,5890,ENSG00000134222,PSRC1,chr1,-,109282679,109282736,109282679,109282731,109283073,...,-0.460,0.000,0.328,0.493,0.000,1.000,1.000,0.000,0.661,A3SS
4,8635,ENSG00000110844,PRPF40B,chr12,+,49633013,49633547,49633426,49633547,49632854,...,0.351,1.000,1.000,0.909,1.000,0.750,0.818,0.158,0.778,A3SS
5,506,ENSG00000197530,MIB2,chr1,+,1625545,1626754,1626649,1626754,1625285,...,-0.138,0.767,0.745,0.906,1.000,1.000,0.972,1.000,1.000,A3SS
6,2770,ENSG00000113108,APBB3,chr5,-,140561586,140561722,140561586,140561701,140561843,...,-0.563,0.469,0.469,0.000,0.227,1.000,0.638,1.000,0.779,A3SS


In [4]:
## Calculate the length of upstream,dowstream exon and intron, and flanked exon, 
## and calcualte the genome coodinates in order to download from NCBI
a3ss_sig_as_coord_len = pd.DataFrame(columns = ['ID','GeneID','geneSymbol','chr','strand',
                                                'seq_1_in_co1','seq_1_in_co2','seq_1_in_co3','seq_1_in_co4',
                                                'seq_1_ex_co1','seq_1_ex_co2','seq_1_ex_co3','seq_1_ex_co4',
                                                'seq_2_in_co1','seq_2_in_co2','seq_2_in_co3','seq_2_in_co4',
                                                'seq_2_ex_co1','seq_2_ex_co2','seq_2_ex_co3','seq_2_ex_co4'])
for i in range(0,len(a3ss_sig)):
    if a3ss_sig['strand'][i] == '-':
        a3ss_sig_as_coord_len.loc[i,'seq_2_ex_co3'] = a3ss_sig.loc[i,'shortEE'].copy()
        a3ss_sig_as_coord_len.loc[i,'seq_2_ex_co2'] = a3ss_sig.loc[i,'shortEE'].copy() + 1
        a3ss_sig_as_coord_len.loc[i,'seq_2_in_co3'] = a3ss_sig.loc[i,'longExonEnd'].copy()
        a3ss_sig_as_coord_len.loc[i,'seq_2_in_co2'] = a3ss_sig.loc[i,'longExonEnd'].copy() + 1
        a3ss_sig_as_coord_len.loc[i,'seq_1_ex_co3'] = a3ss_sig.loc[i,'flankingES'].copy() - 1
        a3ss_sig_as_coord_len.loc[i,'seq_1_ex_co2'] = a3ss_sig.loc[i,'flankingES'].copy()
        a3ss_sig_as_coord_len.loc[i,'seq_1_in_co3'] = a3ss_sig.loc[i,'flankingES'].copy() - 1
        a3ss_sig_as_coord_len.loc[i,'seq_1_in_co2'] = a3ss_sig.loc[i,'flankingES'].copy()
        
        
        if (a3ss_sig.loc[i,'shortEE'] - a3ss_sig.loc[i,'shortES'] + 1) >= 150:
            a3ss_sig_as_coord_len.loc[i,'seq_2_ex_co4'] = a3ss_sig.loc[i,'shortEE'].copy() - 150 + 1
        else:
            a3ss_sig_as_coord_len.loc[i,'seq_2_ex_co4'] = a3ss_sig.loc[i,'shortES'].copy()
        
        if (a3ss_sig.loc[i,'longExonEnd'] - a3ss_sig.loc[i,'longExonStart_0base'] + 1) >= 150:
            a3ss_sig_as_coord_len.loc[i,'seq_1_in_co4'] = a3ss_sig.loc[i,'longExonEnd'].copy() - 150 + 1
        else:
            a3ss_sig_as_coord_len.loc[i,'seq_1_in_co4'] = a3ss_sig.loc[i,'longExonStart_0base'].copy()
        
        if (a3ss_sig.loc[i,'flankingES'] - a3ss_sig.loc[i,'shortEE'] - 1) >= 300:
            a3ss_sig_as_coord_len.loc[i,'seq_2_ex_co1'] = a3ss_sig.loc[i,'shortEE'].copy() + 150
            a3ss_sig_as_coord_len.loc[i,'seq_1_ex_co4'] = a3ss_sig.loc[i,'flankingES'].copy() - 150
        else:
            a3ss_sig_as_coord_len.loc[i,'seq_2_ex_co1'] = a3ss_sig.loc[i,'shortEE'].copy() + int((a3ss_sig.loc[i,'flankingES'] - a3ss_sig.loc[i,'shortEE'] - 1)/2)
            a3ss_sig_as_coord_len.loc[i,'seq_1_ex_co4'] = a3ss_sig.loc[i,'flankingES'].copy() - int((a3ss_sig.loc[i,'flankingES'] - a3ss_sig.loc[i,'shortEE'] - 1)/2)
            
        if (a3ss_sig.loc[i,'flankingES'] - a3ss_sig.loc[i,'longExonEnd'] - 1) >= 300:
            a3ss_sig_as_coord_len.loc[i,'seq_2_in_co1'] = a3ss_sig.loc[i,'longExonEnd'].copy() + 150
            a3ss_sig_as_coord_len.loc[i,'seq_1_in_co4'] = a3ss_sig.loc[i,'flankingES'].copy() - 150
        else:
            a3ss_sig_as_coord_len.loc[i,'seq_2_in_co1'] = a3ss_sig.loc[i,'longExonEnd'].copy() + int((a3ss_sig.loc[i,'flankingES'] - a3ss_sig.loc[i,'longExonEnd'] - 1)/2)
            a3ss_sig_as_coord_len.loc[i,'seq_1_in_co4'] = a3ss_sig.loc[i,'flankingES'].copy() - int((a3ss_sig.loc[i,'flankingES'] - a3ss_sig.loc[i,'longExonEnd'] - 1)/2)

        if (a3ss_sig.loc[i,'flankingEE'] - a3ss_sig.loc[i,'flankingES'] + 1) >= 150:
            a3ss_sig_as_coord_len.loc[i,'seq_1_ex_co1'] = a3ss_sig.loc[i,'flankingES'].copy() + 150 - 1
            a3ss_sig_as_coord_len.loc[i,'seq_1_in_co1'] = a3ss_sig.loc[i,'flankingES'].copy() + 150 - 1
        else:
            a3ss_sig_as_coord_len.loc[i,'seq_1_ex_co1'] = a3ss_sig.loc[i,'flankingEE'].copy()
            a3ss_sig_as_coord_len.loc[i,'seq_1_in_co1'] = a3ss_sig.loc[i,'flankingEE'].copy()
            
    else:
        a3ss_sig_as_coord_len.loc[i,'seq_2_ex_co3'] = a3ss_sig.loc[i,'shortES'].copy()
        a3ss_sig_as_coord_len.loc[i,'seq_2_ex_co2'] = a3ss_sig.loc[i,'shortES'].copy() - 1
        a3ss_sig_as_coord_len.loc[i,'seq_2_in_co3'] = a3ss_sig.loc[i,'longExonStart_0base'].copy()
        a3ss_sig_as_coord_len.loc[i,'seq_2_in_co2'] = a3ss_sig.loc[i,'longExonStart_0base'].copy() - 1
        
        a3ss_sig_as_coord_len.loc[i,'seq_1_ex_co3'] = a3ss_sig.loc[i,'flankingEE'].copy() + 1
        a3ss_sig_as_coord_len.loc[i,'seq_1_ex_co2'] = a3ss_sig.loc[i,'flankingEE'].copy()
        a3ss_sig_as_coord_len.loc[i,'seq_1_in_co3'] = a3ss_sig.loc[i,'flankingEE'].copy() + 1
        a3ss_sig_as_coord_len.loc[i,'seq_1_in_co2'] = a3ss_sig.loc[i,'flankingEE'].copy()
        
        
        if (a3ss_sig.loc[i,'shortEE'] - a3ss_sig.loc[i,'shortES'] + 1) >= 150:
            a3ss_sig_as_coord_len.loc[i,'seq_2_ex_co4'] = a3ss_sig.loc[i,'shortES'].copy() + 150 - 1
        else:
            a3ss_sig_as_coord_len.loc[i,'seq_2_ex_co4'] = a3ss_sig.loc[i,'shortEE'].copy()
        
        if (a3ss_sig.loc[i,'longExonEnd'] - a3ss_sig.loc[i,'longExonStart_0base'] + 1) >= 150:
            a3ss_sig_as_coord_len.loc[i,'seq_2_in_co4'] = a3ss_sig.loc[i,'longExonStart_0base'].copy() + 150 - 1
        else:
            a3ss_sig_as_coord_len.loc[i,'seq_2_in_co4'] = a3ss_sig.loc[i,'longExonEnd'].copy()
            
        
        if (a3ss_sig.loc[i,'shortES'] - a3ss_sig.loc[i,'flankingEE'] - 1) >= 300:
            a3ss_sig_as_coord_len.loc[i,'seq_2_ex_co1'] = a3ss_sig.loc[i,'shortES'].copy() - 150
            a3ss_sig_as_coord_len.loc[i,'seq_1_ex_co4'] = a3ss_sig.loc[i,'flankingEE'].copy() + 150
        else:
            a3ss_sig_as_coord_len.loc[i,'seq_2_ex_co1'] = a3ss_sig.loc[i,'shortES'].copy() - int((a3ss_sig.loc[i,'shortES'] - a3ss_sig.loc[i,'flankingEE'] - 1)/2)
            a3ss_sig_as_coord_len.loc[i,'seq_1_ex_co4'] = a3ss_sig.loc[i,'flankingEE'].copy() + int((a3ss_sig.loc[i,'shortES'] - a3ss_sig.loc[i,'flankingEE'] - 1)/2)
            
        if (a3ss_sig.loc[i,'longExonStart_0base'] - a3ss_sig.loc[i,'flankingEE'] - 1) >= 300:
            a3ss_sig_as_coord_len.loc[i,'seq_2_in_co1'] = a3ss_sig.loc[i,'longExonStart_0base'].copy() - 150
            a3ss_sig_as_coord_len.loc[i,'seq_1_in_co4'] = a3ss_sig.loc[i,'flankingEE'].copy() + 150
        else:
            a3ss_sig_as_coord_len.loc[i,'seq_2_in_co1'] = a3ss_sig.loc[i,'longExonStart_0base'].copy() - int((a3ss_sig.loc[i,'longExonStart_0base'] - a3ss_sig.loc[i,'flankingEE'] - 1)/2)
            a3ss_sig_as_coord_len.loc[i,'seq_1_in_co4'] = a3ss_sig.loc[i,'flankingEE'].copy() + int((a3ss_sig.loc[i,'longExonStart_0base'] - a3ss_sig.loc[i,'flankingEE'] - 1)/2)

        if (a3ss_sig.loc[i,'flankingEE'] - a3ss_sig.loc[i,'flankingES'] + 1) >= 150:
            a3ss_sig_as_coord_len.loc[i,'seq_1_ex_co1'] = a3ss_sig.loc[i,'flankingEE'].copy() - 150 + 1
            a3ss_sig_as_coord_len.loc[i,'seq_1_in_co1'] = a3ss_sig.loc[i,'flankingEE'].copy() - 150 + 1
        else:
            a3ss_sig_as_coord_len.loc[i,'seq_1_ex_co1'] = a3ss_sig.loc[i,'flankingES'].copy()
            a3ss_sig_as_coord_len.loc[i,'seq_1_in_co1'] = a3ss_sig.loc[i,'flankingES'].copy()
            


a3ss_sig_as_coord_len.iloc[:,:5] = a3ss_sig.iloc[:,:5].copy()
a3ss_sig_as_coord_len

,ID,GeneID,geneSymbol,chr,strand,seq_1_in_co1,seq_1_in_co2,seq_1_in_co3,seq_1_in_co4,seq_1_ex_co1,...,seq_1_ex_co3,seq_1_ex_co4,seq_2_in_co1,seq_2_in_co2,seq_2_in_co3,seq_2_in_co4,seq_2_ex_co1,seq_2_ex_co2,seq_2_ex_co3,seq_2_ex_co4
0,6302,ENSG00000144868,TMEM108,chr3,+,133045901,133046020,133046021,133046170,133045901,...,133046021,133046170,133229115,133229264,133229265,133229351,133229119,133229268,133229269,133229351
1,7536,ENSG00000130600,H19,chr11,-,1995998,1995875,1995874,1995835,1995998,...,1995874,1995832,1995834,1995795,1995794,NaN,1995831,1995789,1995788,1995639
2,2594,ENSG00000141994,DUS3L,chr19,-,5787171,5787060,5787059,5786990,5787171,...,5787059,5786953,5786989,5786920,5786919,NaN,5786952,5786846,5786845,5786748
3,5890,ENSG00000134222,PSRC1,chr1,-,109283186,109283073,109283072,109282923,109283186,...,109283072,109282923,109282886,109282737,109282736,NaN,109282881,109282732,109282731,109282679
4,8635,ENSG00000110844,PRPF40B,chr12,+,49632854,49632880,49632881,49632946,49632854,...,49632881,49633030,49632947,49633012,49633013,49633162,49633276,49633425,49633426,49633547
5,506,ENSG00000197530,MIB2,chr1,+,1625285,1625428,1625429,1625486,1625285,...,1625429,1625578,1625487,1625544,1625545,1625694,1626499,1626648,1626649,1626754
6,2770,ENSG00000113108,APBB3,chr5,-,140561849,140561843,140561842,140561783,140561849,...,140561842,140561773,140561782,140561723,140561722,NaN,140561771,140561702,140561701,140561586
7,1218,ENSG00000134802,SLC43A3,chr11,-,57426652,57426587,57426586,57426489,57426652,...,57426586,57426470,57426487,57426390,57426389,NaN,57426469,57426353,57426352,57426203
8,8101,ENSG00000058272,PPP1R12A,chr12,-,79807330,79807225,79807224,79807075,79807330,...,79807224,79807075,79806483,79806334,79806333,NaN,79806447,79806298,79806297,79806165
9,819,ENSG00000116641,DOCK7,chr1,-,62487452,62487397,62487396,62487247,62487452,...,62487396,62487247,62477984,62477835,62477834,NaN,62477975,62477826,62477825,62477699
